In [86]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [87]:
filePath = "../data/hurricane_vessel_dataset_unique.csv"
df = pd.read_csv(filePath)

In [88]:
df.head()

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,...,50-knot Wind Radii SE,50-knot Wind Radii SW,50-knot Wind Radii NW,64-knot Wind Radii NE,64-knot Wind Radii SE,64-knot Wind Radii SW,64-knot Wind Radii NW,Speed mph,hurricane_datetime,PathChange
0,255805914.0,2020-09-01 00:00:00,34.70562,-75.60953,16.2,52.8,55.0,ITAL CONTESSA,IMO9293806,CQIB6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,2020-09-01 00:00:00,veered off course
1,636017112.0,2020-09-01 00:00:00,39.42750,-75.52683,14.7,318.0,319.0,POMERENIA SKY,IMO9339583,A8MG6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,2020-09-01 00:00:00,veered off course
2,249922000.0,2020-09-01 00:00:00,41.58462,-71.28670,0.0,143.2,511.0,OLIVA,IMO9413705,9HA2075,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,2020-09-01 00:00:00,stayed on course
3,220379000.0,2020-09-01 00:00:00,36.92358,-75.92110,6.2,305.6,302.0,GUDRUN MAERSK,IMO9302877,OYAU2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,2020-09-01 00:00:00,veered off course
4,366235000.0,2020-09-01 00:00:00,32.84996,-79.93852,0.0,0.0,58.0,CAPE DOMINGO,IMO7229423,WMHV,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,2020-09-01 00:00:00,veered off course


In [89]:
df.shape

(1866, 42)

In [90]:
df.isna().sum()

MMSI                      0
BaseDateTime              0
LAT                       0
LON                       0
SOG                       0
COG                       0
Heading                   0
VesselName                0
IMO                       0
CallSign                  0
VesselType                0
Status                    0
Length                    0
Width                     0
Draft                     0
Cargo                     0
TransceiverClass          0
Name                      0
Num Entries               0
Year                      0
Time                      0
Record Identifier         0
Status of System          0
Latitude                  0
Longitude                 0
Maximum Sustained Wind    0
Minimum Pressure          0
34-knot Wind Radii NE     0
34-knot Wind Radii SE     0
34-knot Wind Radii SW     0
34-knot Wind Radii NW     0
50-knot Wind Radii NE     0
50-knot Wind Radii SE     0
50-knot Wind Radii SW     0
50-knot Wind Radii NW     0
64-knot Wind Radii N

In [91]:
df.dtypes

MMSI                      float64
BaseDateTime               object
LAT                       float64
LON                       float64
SOG                       float64
COG                       float64
Heading                   float64
VesselName                 object
IMO                        object
CallSign                   object
VesselType                float64
Status                    float64
Length                    float64
Width                     float64
Draft                     float64
Cargo                     float64
TransceiverClass           object
Name                       object
Num Entries                 int64
Year                       object
Time                        int64
Record Identifier          object
Status of System           object
Latitude                  float64
Longitude                 float64
Maximum Sustained Wind    float64
Minimum Pressure          float64
34-knot Wind Radii NE     float64
34-knot Wind Radii SE     float64
34-knot Wind R

In [92]:
df["MMSI"].nunique()

1866

In [93]:
le = LabelEncoder()

# Label Encoding for the columns with string values
df['PathChange'] = le.fit_transform(df['PathChange']) #Encode the PathChange column 1: veered off course, 0: stayed on course
df["TransceiverClass"] = le.fit_transform(df["TransceiverClass"])
df["Status of System"] = le.fit_transform(df["Status of System"])

In [94]:
# Drop datetime, unique identfiers and vessel/hurricane name columns from the dataframe
df.drop(['BaseDateTime', "hurricane_datetime", "Record Identifier", "Name", "Year", "VesselName", "IMO", "Time", "CallSign"], axis=1, inplace=True)

In [95]:
df.dtypes

MMSI                      float64
LAT                       float64
LON                       float64
SOG                       float64
COG                       float64
Heading                   float64
VesselType                float64
Status                    float64
Length                    float64
Width                     float64
Draft                     float64
Cargo                     float64
TransceiverClass            int64
Num Entries                 int64
Status of System            int64
Latitude                  float64
Longitude                 float64
Maximum Sustained Wind    float64
Minimum Pressure          float64
34-knot Wind Radii NE     float64
34-knot Wind Radii SE     float64
34-knot Wind Radii SW     float64
34-knot Wind Radii NW     float64
50-knot Wind Radii NE     float64
50-knot Wind Radii SE     float64
50-knot Wind Radii SW     float64
50-knot Wind Radii NW     float64
64-knot Wind Radii NE     float64
64-knot Wind Radii SE     float64
64-knot Wind R

In [96]:
df.shape

(1866, 33)

In [97]:
df.head()

,MMSI,LAT,LON,SOG,COG,Heading,VesselType,Status,Length,Width,...,50-knot Wind Radii NE,50-knot Wind Radii SE,50-knot Wind Radii SW,50-knot Wind Radii NW,64-knot Wind Radii NE,64-knot Wind Radii SE,64-knot Wind Radii SW,64-knot Wind Radii NW,Speed mph,PathChange
0,255805914.0,34.70562,-75.60953,16.2,52.8,55.0,70.0,0.0,332.0,42.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,1
1,636017112.0,39.42750,-75.52683,14.7,318.0,319.0,70.0,0.0,208.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,1
2,249922000.0,41.58462,-71.28670,0.0,143.2,511.0,70.0,15.0,225.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,0
3,220379000.0,36.92358,-75.92110,6.2,305.6,302.0,70.0,0.0,367.0,42.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,1
4,366235000.0,32.84996,-79.93852,0.0,0.0,58.0,70.0,5.0,207.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.811929,1


In [98]:
df.to_csv("../data/hurricane_vessel_dataset_unique_encoded.csv", index=False)